# Mixins and Validation Hooks

SurrealEngine provides developer-ergonomic features for automatically handling timestamps, soft-deletions, and document-level validation hooks before records are saved to the database. These features make maintaining large models much easier.

In [ ]:
import asyncio
from surrealengine import (
    Document, StringField, IntField, 
    TimestampMixin, SoftDeleteMixin, 
    create_connection
)
from surrealengine.logging import logger
logger.set_level(30) # WARNING

db = create_connection(
    url="memory://",
    namespace="test_ns",
    database="test_mixins",
    make_default=True,
    async_mode=True
)
await db.connect()

## 1. `.clean()` Validation Hooks

The `.clean()` method on any `Document` is automatically called right before `.save()` or `.save_sync()`. This is the perfect place to put cross-field validation, normalization, or default formatting.

In [ ]:
class UserRegistration(Document):
    username = StringField(required=True)
    email = StringField(required=True)
    age = IntField()

    def clean(self):
        # Always call super().clean() first!
        super().clean()
        
        # 1. Normalization
        if self.email:
            self.email = self.email.lower().strip()
            
        # 2. Validation
        if self.age is not None and self.age < 18:
            raise ValueError("Users must be 18 or older to register.")
            
await UserRegistration.create_table(schemafull=True)

In [ ]:
# This will trigger the age validation error
try:
    kid = UserRegistration(username="timmy", email="Timmy@Example.com", age=12)
    await kid.save()
except ValueError as e:
    print(f"Validation Failed: {e}")

In [ ]:
# This will succeed and normalize the email
adult = UserRegistration(username="john_doe", email="  JOHN.DOE@EXAMPLE.COM  ", age=25)
await adult.save()
print(f"Saved email normalized: '{adult.email}'")

## 2. Default Mixins: Timestamps & SoftDeletes

SurrealEngine provides two built-in Mixins that leverage the `.clean()` hook to automatically manage boilerplate fields for you:
- `TimestampMixin`: Automatically injects `created_at` and `updated_at` properties, mapping to `time::now()` on save.
- `SoftDeleteMixin`: Introduces non-destructive `deleted_at` tracking and overrides `.delete()`.

In [ ]:
class BlogPost(TimestampMixin, SoftDeleteMixin, Document):
    title = StringField(required=True)
    content = StringField()

await BlogPost.create_table(schemafull=True)

In [ ]:
post = BlogPost(title="My First Post", content="Hello world!")
await post.save()

print(f"Created At: {post.created_at}")
print(f"Updated At: {post.updated_at}")
print(f"Deleted At: {post.deleted_at}")

In [ ]:
import asyncio

# Wait a moment so we can see the timestamp change
await asyncio.sleep(1)

post.content = "Hello world, updated!"
await post.save()

print(f"New Updated At: {post.updated_at}")

### Soft Deletes

Calling `.delete()` on a model inheriting from `SoftDeleteMixin` will not actually remove it from the database! Instead, it will populate the `deleted_at` field and save.

In [ ]:
# Soft delete the post
await post.delete()

print(f"Deleted At is now populated: {post.deleted_at}")

# Let's verify it still exists in the database
raw_results = await db.client.query("SELECT * FROM blog_post")
count = len(raw_results[0]['result'])
print(f"Posts actually in DB: {count}")

In [ ]:
# If we need to permanently delete it, we can use hard_delete()
await post.hard_delete()

raw_results = await db.client.query("SELECT * FROM blog_post")
count = len(raw_results[0]['result'])
print(f"Posts in DB after hard delete: {count}")

In [ ]:
await db.disconnect()